# **Trainig and using YOLOv8 for road surface segmentation**

### Installing required packages

In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet ultralytics
!pip install --quiet torch torchvision torchaudio

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available

### Initial variables setting

In [ ]:
# If you want to use a previously trained model
I_want_to_use_an_already_trained_model = True
path_to_the_trained_yolov8_model = 'models/yolov8_road_segmentation/weights/best.pt'


# If you want to train a YOLOv8 model
dataset_dir = '/home/nibio/my_desk/roadsens/model_training/datasets/roadsens/roadsens_published_dataset/'
data_yaml=f"{dataset_dir}/data.yaml"
dir_name_to_save_the_results= "yolov8_road_segmentation"

yolov8_model_size = "nano"        # options: ["nano", "small", "medium", "large", "xlarge"]
task_of_the_model = "segment"     # options: ["detect", "segment"]

treat_all_classes_as_one_class = True


In [ ]:
from ultralytics import YOLO

if I_want_to_use_an_already_trained_model:
  try:
    model = YOLO(path_to_the_trained_yolov8_model)
    print(f"The model: {path_to_the_trained_yolov8_model} loaded.")
  except:
    print(f"Could not load the model: {path_to_the_trained_yolov8_model}.\nA YOLOv8 {yolov8_model_size} will be used.")
    if task_of_the_model.lower() == "segment":
      model = YOLO(f"yolov8{yolov8_model_size[0].lower()}-seg.yaml")
    elif (task_of_the_model.lower() == "detect"):
      model = YOLO(f"yolov8{yolov8_model_size[0].lower()}.yaml") 
else:
  if task_of_the_model.lower() == "segment":
    model = YOLO(f"yolov8{yolov8_model_size[0].lower()}-seg.yaml") 
  elif (task_of_the_model.lower() == "detect"):
    model = YOLO(f"yolov8{yolov8_model_size[0].lower()}.yaml") 


### <a name="yolo_training_params"></a>Training the model


In [ ]:
image_size = 640
batch = 12 
epochs=2000  

intersect_over_union_for_accepting_a_detection = 0.95 
use_dataset_augmentation = True 

model.train(data=data_yaml,
            imgsz=image_size,
            batch=batch,
            epochs=epochs,
            name=dir_name_to_save_the_results,
            exist_ok=False,
            iou=intersect_over_union_for_accepting_a_detection,
            single_cls=treat_all_classes_as_one_class,
            augment=use_dataset_augmentation,
            translate= 0.5,
            scale= 0.9,
            mixup=0.3) 


**Validating the trained model on the test dataset**

In [ ]:
metrics = model.val(split="test",data=data_yaml, single_cls=treat_all_classes_as_one_class, name=dir_name_to_save_the_results+"_on_test_dataset")

<br>

### Using the model to predict

In [ ]:
import os

file_or_folder_to_apply_the_model_for_prediction = "/home/nibio/my_desk/roadsens/model_training/datasets/roadsens/roadsens_published_dataset/test/images" 

if os.path.exists(file_or_folder_to_apply_the_model_for_prediction):
  res = model.predict(source=file_or_folder_to_apply_the_model_for_prediction,
                      save=True,
                      save_txt=True, show_labels=False, line_width=1)